<a href="https://colab.research.google.com/github/davis689/binder/blob/master/CHEM461/boundary_conditions_and_normalization_of_PIB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Particle-in-a-box
## Symbolic manipulation of the wavefunction
Here we will practice using Jupyter notebooks and Python to symbolically manipulate equations. The package that allows this functionality is called SymPy.
### Applying the boundary and normalization conditions
For the particle in a box that is contrained to be between  x =0 and  x = L , assume that we have found an initial wavefunction,
ψ=Asin(nπx/L)+Bcos(nπx/L)
This solution is generic in that  A,B , and  k  are arbitrary. We first apply the boundary conditions to begin determining their values that apply to our specific solution. Let's first set up the problem. Since we'll be solving this symbolically, set up the symbols we want to use in the wavefunction. Some of them are going to be positive so if we specify that, it may be easier to solve in some cases. Then we'll set up wavefunction expression using sp.Eq(). Inside the parentheses we put the two sides of the equation separated by a comma. Then display the results.

In [1]:
# import and initialize libraries
import matplotlib.pyplot as plt # for plotting if we want to do it.

import sympy as sp # deal with symbolic calculations
from sympy import oo, E, I # specifically import infinity, exponential, and complex numbers so we don't have to do sp.oo and sp.E and sp.I

sp.init_printing() # make equations look nice

In [2]:
x=sp.var('x', real=True) # x is the variable and it's real.
A, k  = sp.symbols('A k',positive=True,nonzero=True) # assume that A, k and L are positive
L=sp.symbols('L',real=True,nonzero=True,positive=True) # The length of the box is real, positive, and nonzero
psi,B=sp.symbols('\\psi,B') # no assumption about these
n=sp.symbols('n',positive=True,integer=True,nonzero=True) # the quantum number is positive and an integer and nonzero

# not all the assumptions (real, positive, nonzero, etc.) are necessary here. x,A,k,L,psi,B,n='sp.symbols('x A k L psi B n') may work almost the same.
# They do help Sympy to know how to simplify expressions. So where you know what to expect from a constant, it's useful to classify them this way.

In [3]:
solution_ABk=sp.Eq(psi,A*sp.sin(k*x)+B*sp.cos(k*x)) #The name is meant to indicate that the solution includes A, B, and k constants
solution_ABk

\psi = A⋅sin(k⋅x) + B⋅cos(k⋅x)

If we need to access one side or the other we can do that with ```.rhs``` or ```.lhs``` for right-hand-side and left-hand-side.

In [4]:
solution_ABk.lhs

\psi

In [5]:
solution_ABk.rhs

A⋅sin(k⋅x) + B⋅cos(k⋅x)

## First boundary condition: $\psi=0$ at $x=0$.
We apply the first boundary condition. Since the wavefunction outside the box is zero, the wavefunction inside the box must go to zero at the boundaries. First we apply the boundary condition at $x=0$. To substitute $x=0$ into our equation (without actually changing it's stored form), use ```sp.subs()```.

In [6]:
solution_ABk.subs(x,0).subs(psi,0) # for multiple subsitutions there is an alternative form of .subs.
# solutions.subs({x:0,psi:0}) would do the same thing with only one .subs.

0 = B

We'll update our solution to reflect this.

In [7]:
solution_Ak=solution_ABk.subs(B,0) # solution with B=0 substituted. The new name doesn't include B.
solution_Ak

\psi = A⋅sin(k⋅x)

# Second boundary condition: $\psi=0$ at $x=L$
This solution is more specific now that we have solved for $B$ but it still has two constants to be determined. To solve for another one, substitute $x=L$ and solve for $A$. The ```sp.solve()``` routine needs two inputs. The first is the equation solved so that one side is zero. The second is the variable that you want to solve for. The solve routine returns a list of solutions. Since we want the only solution, we can append [0] to get the first (and only) solution itself instead of a one element list.

In [8]:
sp.solve(solution_Ak.subs([(x,L),(psi,0)]),k)[0] #solve for k if psi=0 and x=L

π
─
L

Now the program has given us one solution but really any integer multiplied by this is also a solution. So we'll substitute $n\pi/L$ for $k$ instead. Here we update our wavefunction with the value of $k$.

In [9]:
solution_A=solution_Ak.subs(k,n*sp.pi/L) #solution with k substituted and a new name not including k
solution_A

            ⎛π⋅n⋅x⎞
\psi = A⋅sin⎜─────⎟
            ⎝  L  ⎠

## Normalization
Now we need to solve for $A$. We are out of boundary conditions to apply. Here we rely on the normalization condition. The integral of $|\psi|^2$ must be equal to 1 if integrated over the whole of space. Here that is from 0 to $L$.

In [10]:
sp.integrate((solution_A.rhs**2),(x,0,L)) #integrate the square of the wavefunction


 2  
A ⋅L
────
 2  

### Complex Conjugates
In our normalization above, we elided over something that we correct here. The probability is not really the square of the wavefunction, it is the product of the complex conjugate of our wavefunction with the wavefunction itself. Here in our case there is no difference because our wavefunction is real. But in general we need to be careful just using the square of the wavefunction.

Fortunately, sympy (sp), has the ability to take the complex conjugate. ```sp.conjugate()``` will show us the complex conjugate of the right-hand-side of the wavefunction. Is it the same as the wavefunction?

In [11]:
sp.conjugate(solution_A.rhs)

     ⎛π⋅n⋅x⎞
A⋅sin⎜─────⎟
     ⎝  L  ⎠

Assume, for a moment, our wavefunction were $e^{ikx}$. Show the complex conjugate of this wavefunction.

In [14]:
psi1=E**(I*k*x)


 -ⅈ⋅k⋅x
ℯ      

### Back to Normalization
We can use the complex conjugate in determining our normalization constant. Integrate of the righthand-side of our solution between 0 and $L$. This gives a result that should be 1 but only will be with the correct value of $A$.



In [15]:
sp.integrate((sp.conjugate(solution_A.rhs))*(solution_A.rhs),(x,0,L))

 2  
A ⋅L
────
 2  

You may be able to figure out the correct value of $A$ to make the above expression equal to 1 but we can code it too. What we need is to set up an equation setting the integral equal to 1. ```sp.Eq(left side of equation,right side of equation)``` is the form that we need. Then we solve that equation for A and choose the first ([0]) solution.

In [16]:
normEq=sp.Eq(sp.integrate((sp.conjugate(solution_A.rhs))*(solution_A.rhs),(x,0,L)),1)
normconst=sp.solve(normEq,A)[0]
normconst

√2
──
√L

So we can now present our final form of the specific solution to the Schrodinger equation that satisifies the boundary conditions of the particle-in-a-box between 0 and $L$ and the normalization condition.

In [17]:
solution=solution_A.subs(A,normconst)
solution

             ⎛π⋅n⋅x⎞
       √2⋅sin⎜─────⎟
             ⎝  L  ⎠
\psi = ─────────────
             √L     

Note that we've been changing the name of our solution at each step. That helps avoid confusion if you end up going back to change things and forget that the solution has been redefined somewhere below and it also makes it possible to recall all of the different versions. ```display()``` is a fancier version of ```print()```.

In [18]:
print('Original generic solution to the SE:')
display(solution_ABk)
print('After application of the first boundary condition:')
display(solution_Ak)
print('After application of the second boundary condition:')

display(solution_A)
print('After application of the normalization condition:')
display(solution)

Original generic solution to the SE:


\psi = A⋅sin(k⋅x) + B⋅cos(k⋅x)

After application of the first boundary condition:


\psi = A⋅sin(k⋅x)

After application of the second boundary condition:


            ⎛π⋅n⋅x⎞
\psi = A⋅sin⎜─────⎟
            ⎝  L  ⎠

After application of the normalization condition:


             ⎛π⋅n⋅x⎞
       √2⋅sin⎜─────⎟
             ⎝  L  ⎠
\psi = ─────────────
             √L     

## Orthonormality
Now let's check the orthonormality of our particle-in-a-box wavefunction. This requires the evaluation of $$\int \psi_i^*\psi_j d\tau$$ If $i=j$ the result should be one (probability is one or perfect overlap of the two wavefunction expressions) and if $i\neq j$ the result should be zero (the two eigenfunctions are 'perpendicular' to each other so don't overlap at all).

Here we'll create two variables, $i$ and $j$, for the quantum numbers of the two eigenfunctions. Adjust these values and evaluate the integral to see how it changes.

In [19]:
i=1
j=1
sp.integrate(sp.conjugate(solution.rhs.subs(n,i))*solution.rhs.subs(n,j),(x,0,L))

1

We could, of course, check all values of quantum numbers in some range systematically. Here we check all combinations of values of $i$ and $j$ between 1 and 5 (remember ```range(1,6)``` makes a list of integers from 1 up to but not including 6).

In [20]:
print('i j integral')
for i in range(1,6):
  for j in range(1,6):
    integ=sp.integrate(sp.conjugate(solution.rhs.subs(n,i))*solution.rhs.subs(n,j),(x,0,L))
    print(i,j,integ)

i j integral
1 1 1
1 2 0
1 3 0
1 4 0
1 5 0
2 1 0
2 2 1
2 3 0
2 4 0
2 5 0
3 1 0
3 2 0
3 3 1
3 4 0
3 5 0
4 1 0
4 2 0
4 3 0
4 4 1
4 5 0
5 1 0
5 2 0
5 3 0
5 4 0
5 5 1


#Do it for yourself
Now you can try this for a different form of the solution of the particle in a box problem. Let's use $\psi=A e^{-ikx}+ B e^{ikx}$. First, satisfy yourself that this is a solution to the Schrödinger equation. Then use the techniques above to determine the value of A, B, and k. Finally show that the solutions are orthonormal.